In [ ]:
import os
import pickle
import numpy as np
import detectron2
import torch
import mrcnn_utils
import mrcnn_visualize
import pathlib
import skimage
import skimage.io
import evaluate

import analyze
import matplotlib.pyplot as plt

In [ ]:
with open('../data/outputs_np.pickle', 'rb') as f:
    pred_data = pickle.load(f)

pred_instances = [analyze.instance_set().read_from_numpy_out(
    x,y,True) for x, y in pred_data.items()]
    
    
gt_json_path = pathlib.Path('../data/raw/via_2.0.8/test_powder.json')
gt_instances = [analyze.instance_set().read_from_ddict(x, True)
             for x in analyze.get_data_dicts(gt_json_path)]

gt_instances = [{x.filepath.name: x for x in gt_instances}[f] 
                for f in pred_data] # hacky but this rearranges 
                # the order of the instances so both sets are the same

In [ ]:
# ground truth
gt = gt_instances[-1]

# predicted
pred = pred_instances[-1]


In [ ]:
# ground truth
colors = evaluate.random_colors(len(gt.class_idx), 32)
mrcnn_visualize.display_instances(gt.img, gt.boxes, gt.masks, 
                                  gt.class_idx, [''], colors=colors)

In [ ]:
## original mask predictions 

pcolors = evaluate.random_colors(len(pred.class_idx), 34)
mrcnn_visualize.display_instances(gt.img, pred.boxes, pred.masks, 
                                  pred.class_idx, [''], colors=pcolors)

In [ ]:
# mask predictions with watershed update
mask_ws, im_new = evaluate.refine_masks_watershed_li(gt.img, pred.masks, 
                                                     True, True, (0,0,0), True)
box_new = mrcnn_utils.extract_bboxes(mask_ws)

mrcnn_visualize.display_instances(im_new, box_new, mask_ws, 
                                  pred.class_idx, [''], colors=pcolors)

In [ ]:
# filter out outlier masks
gt_filt = analyze.filter_mask_size(gt.masks, bbox=gt.boxes, class_idx=gt.class_idx,
                                  colors=np.asarray(colors))
pred_filt = analyze.filter_mask_size(pred.masks, bbox=pred.boxes,
                                            class_idx=pred.class_idx, colors=np.asarray(pcolors))


In [ ]:
matches = analyze.fast_instance_match(gt_filt[0], pred_filt[0])

In [ ]:
matches.keys()

In [ ]:
# verify that instances are matched correctly
i = 50
m_idx = matches['gt_tp'][i]
fig, ax = plt.subplots(1,2)
ax[0].imshow(gt_filt[0][:,:,m_idx[0]])
ax[1].imshow(pred_filt[0][:,:,m_idx[1]])
ax[0].set_title('IOU: {:.3f}'.format(matches['IOU_match'][i]))
fig.tight_layout()

In [ ]:
# distribution of IOU scores for matches
plt.hist(matches['IOU_match'])

In [ ]:
stats = analyze.mask_match_stats(gt_filt[0], pred_filt[0])

In [ ]:
stats['mask_recall'].mean()

In [ ]:
titles = lambda x,y: 'match {0}: {1:.3f}\nmask {0}:'.format(x,y)

fig, ax = plt.subplots(1,2, sharey=True)
ax[0].boxplot(stats['mask_precision'])
ax[0].set_title(titles('precision',stats['match_precision']))

ax[1].boxplot(stats['mask_recall'])
ax[1].set_title(titles('recall',stats['match_recall']))


In [ ]:
np.unique(colors, axis=1).shape